## 垃圾分类样例
**训练过程参考** [MobileNetV2垃圾分类](https://gitee.com/ascend/samples/wikis/MobileNetV2%E5%9E%83%E5%9C%BE%E5%88%86%E7%B1%BB?sort_id=3404387)     
功能：使用mobilenetV2模型对输入图片进行分类推理。  
样例输入：待推理的jpg图片。   
样例输出：推理后的jpg图片。
## 前期准备  
基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可参考如下步骤：

1. 获取模型和测试数据：我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，将模型和测试数据下载到本地，将mobilenetv2.air和insert_op_yuv.cfg放入model目录，将测试图片放入data目录。
2. 模型转换：进入model目录，执行：  
    ```shell
    atc --model=./mobilenetv2.air --framework=1 --output=garbage_yuv --soc_version=Ascend310B1 --insert_op_conf=./insert_op_yuv.cfg --input_shape="data:1,3,224,224" --input_format=NCHW
    ```

    其中各个参数具体含义如下：
    * --model：原始模型文件。
    * --framework：原始框架类型，0:Caffe; 1:MindSpore; 2:Tensorflow; 5:onnx。
    * --output：离线推理om模型文件路径。
    * --soc_version：昇腾AI处理器型号，填写"Ascend310B1"。
    * --input_shape：模型输入节点名称和shape。
    * --input_format：输入Tensor的内存排列方式。
    * --output_type：指定网络输出数据类型或指定某个输出节点的输出类型。
    * --insert_op_conf: 插入算子的配置文件路径与文件名， 例如aipp预处理算子。

## 模型推理实现  

In [ ]:
import sys
import os

import numpy as np
import acl
import base64
import acllite_utils as utils
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

from acllite_imageproc import AclLiteImageProc
import constants as const
from acllite_model import AclLiteModel
from acllite_image import AclLiteImage
from acllite_resource import resource_list


SRC_PATH = os.getcwd() 
MODEL_PATH = os.path.join(SRC_PATH, "model/garbage_yuv.om") # 模型路径
MODEL_WIDTH = 224 # 模型宽度
MODEL_HEIGHT = 224 # 模型高度
image_net_classes = [
   "Seashel", "Lighter","Old Mirror", "Broom","Ceramic Bowl", "Toothbrush","Disposable Chopsticks","Dirty Cloth",
     "Newspaper", "Glassware", "Basketball", "Plastic Bottle", "Cardboard","Glass Bottle", "Metalware", "Hats", "Cans", "Paper",
      "Vegetable Leaf","Orange Peel", "Eggshell","Banana Peel",
    "Battery", "Tablet capsules","Fluorescent lamp", "Paint bucket"] # 分类类别

In [ ]:
#AclLiteResource类，包含初始化acl、释放acl资源功能
class AclLiteResource:
    """
    AclLiteResource
    """
    def __init__(self, device_id=0):
        self.device_id = device_id
        self.context = None
        self.stream = None
        self.run_mode = None
        
    def init(self):
        """
        init resource
        """
        print("init resource stage:")
        ret = acl.init() # acl初始化

        ret = acl.rt.set_device(self.device_id) # 指定运算的device
        utils.check_ret("acl.rt.set_device", ret)

        self.context, ret = acl.rt.create_context(self.device_id) # 创建context
        utils.check_ret("acl.rt.create_context", ret)

        self.stream, ret = acl.rt.create_stream() # 创建stream
        utils.check_ret("acl.rt.create_stream", ret)

        self.run_mode, ret = acl.rt.get_run_mode() # 获取运行模式
        utils.check_ret("acl.rt.get_run_mode", ret)

        print("Init resource success")

    def __del__(self):
        print("acl resource release all resource")
        resource_list.destroy()
        if self.stream:
            print("acl resource release stream")
            acl.rt.destroy_stream(self.stream) # 销毁stream

        if self.context:
            print("acl resource release context")
            acl.rt.destroy_context(self.context) # 释放context

        print("Reset acl device ", self.device_id)
        acl.rt.reset_device(self.device_id)
        print("Release acl resource success") # 释放device


In [ ]:
# 根据class_id对应的类别 
def get_image_net_class(class_id):
    if class_id >= len(image_net_classes):
        return "unknown"
    else:
        return image_net_classes[class_id]
# 转化图片格式
def pre_process(image, dvpp):
    """
    预处理
    """
    image_input = image.copy_to_dvpp()
    yuv_image = dvpp.jpegd(image_input)

    print("decode jpeg end")
    resized_image = dvpp.resize(yuv_image, 
                    MODEL_WIDTH, MODEL_HEIGHT)

    print("resize yuv end")
    return resized_image

def post_process(infer_output, image_file):
    """
    后处理
    """
    print("post process")
    data = infer_output[0]
    vals = data.flatten()
    top_k = vals.argsort()[-1:-6:-1] # 获取推理出的id
    object_class = get_image_net_class(top_k[0]) # 获取推理出id对应的类别
    output_path = os.path.join(os.path.join(SRC_PATH, "out"), os.path.basename(image_file)) # 拼接输出路径
    origin_image = Image.open(image_file) # 获取原始图片
    draw = ImageDraw.Draw(origin_image) # 绘图
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", size=20) # 设置字体格式
    font.size =50
    draw.text((10, 50), object_class, font=font, fill=255) # 图片上写入类别
    origin_image.save(output_path) # 保存推理出的图片到输出路径
    plt.figure('image') 
    plt.imshow(origin_image)
    plt.show() # 显示推理图片      
    return 

def construct_image_info():
    """构造图像信息"""
    image_info = np.array([MODEL_WIDTH, MODEL_HEIGHT, 
                           MODEL_WIDTH, MODEL_HEIGHT], 
                           dtype = np.float32) 
    return image_info

def main():    
    image_dir = "data" # 推理图片路径
    # acl初始化
    acl_resource = AclLiteResource() 
    acl_resource.init()
    # 加载模型
    model = AclLiteModel(MODEL_PATH)
    # 预处理
    dvpp = AclLiteImageProc(acl_resource)
            
    images_list = [os.path.join(image_dir, img)
                   for img in os.listdir(image_dir)
                   if os.path.splitext(img)[1] in const.IMG_EXT]

    #创建推理输出路径，
    if not os.path.isdir(os.path.join(SRC_PATH, "out")):
        os.mkdir(os.path.join(SRC_PATH, "out"))

    image_info = construct_image_info()
    # 循环推理图片
    for image_file in images_list:   
        # 进行预处理
        image = AclLiteImage(image_file) 
        resized_image = pre_process(image, dvpp)
        print("pre process end")
        # 执行推理
        result = model.execute([resized_image,])  
        # 进行后处理
        post_process(result, image_file)    

        print("process "+image_file+" end")

In [ ]:
main()

# 样例总结与扩展
1. 初始化acl资源：在调用acl相关资源时，必须先初始化AscendCL，否则可能会导致后续系统内部资源初始化出错。  
2. 对图片进行前处理：在此样例中，我们首先根据图片路径，构造AclLiteImage类型的数据。注意由于模型输入是yuv格式，所以我们利用了AclLiteImageProc.jpegd将图片转为yuv，使得模型正常推理。  
3. 推理：利用AclLiteModel.execute接口对图片进行推理。    
4. 对推理结果进行后处理：将推理出的分类类别画在图片上  
5. 可视化图片：利用plt将结果画出。
6. 保存图片到output目录: 利用Image将图片保存在输出目录。